# Tokenizer

Lo normal es usar librerias que me permitan usar los tokenizers. Aqui usamos Tensorflow pero Hugginface tiene otros tokens.  

Hay tokens que son separators

In [1]:
sentences = [
    "Me gusta la tortilla de patatas",
    "La tortilla de champiñones es mucho mas jugosa"
]

In [2]:
sentences[0].split(" ")

['Me', 'gusta', 'la', 'tortilla', 'de', 'patatas']

Para importar el tokenizer, es un objeto, lo importo. El fit asocia numeros a mis palabras. Va a crear su propio vocabulario/diccionario.  


## Siempre debo tener un diccionario mapeado, un numero por palabras.  
Cuando entrene algoritmos tendre 10M de palabras

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

In [4]:
""" diccionario """

tokenizer.word_index
tokenizer.index_word

{1: 'la',
 2: 'tortilla',
 3: 'de',
 4: 'me',
 5: 'gusta',
 6: 'patatas',
 7: 'champiñones',
 8: 'es',
 9: 'mucho',
 10: 'mas',
 11: 'jugosa'}

In [5]:
sentences[0]

'Me gusta la tortilla de patatas'

## Ahora vamos a limitar el vocabulario

In [6]:
tokenizer.texts_to_sequences(sentences)

[[4, 5, 1, 2, 3, 6], [1, 2, 3, 7, 8, 9, 10, 11]]

In [7]:
# limitamos el vocabulario

tam_vocab = 4
tokenizer = Tokenizer(num_words=tam_vocab, oov_token='<OOV>')
tokenizer.fit_on_texts(sentences)
tokenizer.texts_to_sequences(sentences)

[[1, 1, 2, 3, 1, 1], [2, 3, 1, 1, 1, 1, 1, 1]]

In [8]:
tokenizer.word_index

{'<OOV>': 1,
 'la': 2,
 'tortilla': 3,
 'de': 4,
 'me': 5,
 'gusta': 6,
 'patatas': 7,
 'champiñones': 8,
 'es': 9,
 'mucho': 10,
 'mas': 11,
 'jugosa': 12}

# Padding

Si tengo una secuencia con 6 palabras y otra con 8, ajusto todas las secuencias al maximo de las 2, a 8 por ejemplo, o al minimo por ejemplo, o a 7. Por que se usa? En una red neuronal, fijo un tamano fijo siempre de entrada, por lo que le meto la longitud de la frase y lo relleno con 0s. (0,0,0,1,3,2,0,0)

In [9]:
tam_vocab = 100
tokenizer = Tokenizer(num_words=tam_vocab, oov_token='<OOV>')
tokenizer.fit_on_texts(sentences)
tokenized_sentences = tokenizer.texts_to_sequences(sentences)
tokenized_sentences

[[5, 6, 2, 3, 4, 7], [2, 3, 4, 8, 9, 10, 11, 12]]

In [10]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

pad_sequences(tokenized_sentences)

array([[ 0,  0,  5,  6,  2,  3,  4,  7],
       [ 2,  3,  4,  8,  9, 10, 11, 12]])

In [11]:
sentences = [
    "Me gusta la tortilla de patatas",
    "La tortilla de champiñones es mucho mas jugosa",
    "Los champiñones con jamon y un poco de vinagre saben genial"
]

In [12]:
tam_vocab = 100
tokenizer = Tokenizer(num_words=tam_vocab, oov_token='<OOV>')
tokenizer.fit_on_texts(sentences)
tokenized_sentences = tokenizer.texts_to_sequences(sentences)
tokenized_sentences
pad_sequences(tokenized_sentences)

array([[ 0,  0,  0,  0,  0,  6,  7,  3,  4,  2,  8],
       [ 0,  0,  0,  3,  4,  2,  5,  9, 10, 11, 12],
       [13,  5, 14, 15, 16, 17, 18,  2, 19, 20, 21]])

In [13]:
pad_sequences(tokenized_sentences, maxlen=5)

array([[ 7,  3,  4,  2,  8],
       [ 5,  9, 10, 11, 12],
       [18,  2, 19, 20, 21]])

In [14]:
pad_sequences(tokenized_sentences, truncating='post', maxlen=5)
pad_sequences(tokenized_sentences, truncating='pre', maxlen=5)

array([[ 7,  3,  4,  2,  8],
       [ 5,  9, 10, 11, 12],
       [18,  2, 19, 20, 21]])

In [15]:
pad_sequences(tokenized_sentences, padding='post')
pad_sequences(tokenized_sentences, padding='pre')

array([[ 0,  0,  0,  0,  0,  6,  7,  3,  4,  2,  8],
       [ 0,  0,  0,  3,  4,  2,  5,  9, 10, 11, 12],
       [13,  5, 14, 15, 16, 17, 18,  2, 19, 20, 21]])

# Stemming

Stemming  
De Me -> me, de gusta -> gust, de mucho -> much.  
Se queda con la raiz, tiene sentido, te ahorras vocabulario. 

Lemmatization 
Me -> yo, gusta -> gustar, la-> el, patatas -> patata.
Para algo generativo esto no me vale, pero para hacer una categorizacion de textos si, por ejemplo

In [16]:
sentences = [
    "Me gusta la tortilla de patatas",
    "La tortilla de champiñones es mucho mas jugosa",
    "Yo marco un gol por minuto",
    "El marco de un cuadro está roto"
]

In [17]:
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer('spanish')

In [18]:
for sentence in sentences:
    for word in sentence.split():
        stemmed_word = stemmer.stem(word)
        print(f"{word} --> {stemmed_word}")

Me --> me
gusta --> gust
la --> la
tortilla --> tortill
de --> de
patatas --> patat
La --> la
tortilla --> tortill
de --> de
champiñones --> champiñon
es --> es
mucho --> much
mas --> mas
jugosa --> jugos
Yo --> yo
marco --> marc
un --> un
gol --> gol
por --> por
minuto --> minut
El --> el
marco --> marc
de --> de
un --> un
cuadro --> cuadr
está --> esta
roto --> rot


# Lemmatizing

In [19]:
import spacy

nlp = spacy.load("es_core_news_sm")

In [20]:
for sentence in sentences:
    doc = nlp(sentence)
    for word in doc:
        lemmatized_word = word.lemma_
        print(f"{word} --> {lemmatized_word}")

Me --> yo
gusta --> gustar
la --> el
tortilla --> tortilla
de --> de
patatas --> patata
La --> el
tortilla --> tortilla
de --> de
champiñones --> champiñón
es --> ser
mucho --> mucho
mas --> mas
jugosa --> jugoso
Yo --> yo
marco --> marcar
un --> uno
gol --> gol
por --> por
minuto --> minuto
El --> el
marco --> marco
de --> de
un --> uno
cuadro --> cuadro
está --> estar
roto --> roto


# Stopwords

Coges los articulos y los eliminas, los filtras, preposiciones, conectores, adv, etc.

In [21]:
stopwords_list = nlp.Defaults.stop_words
print(stopwords_list)

{'atras', 'posible', 'ésos', 'segun', 'podrias', 'ultimo', 'sido', 'cuantos', 'aún', 'cuantas', 'tuyos', 'bien', 'nueva', 'modo', 'demás', 'su', 'todos', 'estos', 'ustedes', 'habrá', 'ningún', 'delante', 'hacerlo', 'mismos', 'muy', 'aquí', 'ya', 'poner', 'buena', 'quienes', 'consigues', 'podrá', 'éstas', 'dice', 'haber', 'tres', 'o', 'vosotros', 'poca', 'hago', 'míos', 'quiza', 'otro', 'aquél', 'nunca', 'tenga', 'ahi', 'consigue', 'llegó', 'ante', 'ningunas', 'podriais', 'supuesto', 'mía', 'tuvo', 'ambos', 'día', 'buen', 'quién', 'nuestra', 'van', 'salvo', 'sabe', 'excepto', 'sean', 'nuevos', 'u', 'he', 'pueda', 'sino', 'proximo', 'tú', 'segundo', 'sabeis', 'estamos', 'sola', 'mio', 'hecho', 'usted', 'parece', 'todavía', 'detrás', 'comentó', 'llevar', 'menudo', 'aproximadamente', 'grandes', 'cuándo', 'existe', 'apenas', 'tener', 'parte', 'hicieron', 'pronto', 'una', 'grande', 'ciertas', 'ellos', 'sabemos', 'fui', 'un', 'tuya', 'afirmó', 'muchos', 'sea', 'mia', 'estados', 'allí', 'cuánt

# Ejemplo de clasificación de textos

## Dataset de Amazon

In [22]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/eduardofc/data/main/amazon_home.csv")
df.head()

,stars,review_body,review_title,product_category
0,1,Jamás me llegó y el vendedor nunca contacto co...,Jamás me llegó,home
1,1,Pone que son 4 piezas y la realidad es que es ...,Mala descripción,home
2,1,Saltan los plomos al tercer día de uso. A devo...,Saltan los plomos al utilizarla,home
3,1,No me ha gustado de hecho la devolví . Súper p...,No merece la pena,home
4,1,"Por más que busque, no le encuentro el agujero...",No tiene agujero para rellenar la piñata,home


In [23]:
df.tail()

,stars,review_body,review_title,product_category
26957,5,"Lo esperado, buena toalla.",Bien,home
26958,5,Tal y como viene explicado en la descripción. ...,Bonito y práctico,home
26959,5,Sartén antiadherente para todo tipo de comidas...,Sartén antiadherente,home
26960,5,"llego de un día para otro, super fácil de mont...",calidad precio,home
26961,5,"Súper bien! Las brochas son buenas, no sé meno...",Preciosas,home


In [24]:
df.shape

(26962, 4)

In [25]:
df.groupby('stars').size()

stars
1    5391
2    5542
3    5477
4    5309
5    5243
dtype: int64

In [26]:
df1 = df[df.stars==1]
df5 = df[df.stars==5]

df = pd.concat([df1, df5])
df.groupby("stars").size()

stars
1    5391
5    5243
dtype: int64

## Creo columna 1 si es 5stars y 0 else

In [27]:
df['good_product'] = df.stars.map(lambda x: 1 if x==5 else 0)
df = df[['review_body', 'good_product']]

## Preprocessing

In [28]:
# quitar las stopwords

from nltk.tokenize import word_tokenize

def remove_stopwords(txt):
    tokens = word_tokenize(txt)
    filtered_tokens = [word for word in tokens if word.lower() not in stopwords_list]
    return ' '.join(filtered_tokens)

txt = 'Jamás me llegó y el vendedor nunca contacto co...'
remove_stopwords(txt)

'Jamás vendedor contacto co ...'

In [29]:
# lemmatizing

def lemmatize(txt):
    doc = nlp(txt)
    lemas = [word.lemma_ for word in doc]
    return ' '.join(lemas)

txt = 'Jamás me llegó y el vendedor nunca contacto co...'
lemmatize(txt)

'jamás yo llegar y el vendedor nunca contactar co ...'

In [30]:
# limpieza con regex

import re

def cleansing(txt):
    return re.sub('[^a-zA-Záéíóúü .,]', '', txt.lower())

txt = 'Jamás me llegó y el vendedor nunca contacto co...'
cleansing(txt)

'jamás me llegó y el vendedor nunca contacto co...'

In [31]:
df['text'] =df.review_body.map(remove_stopwords)
df['text'] =df.text.map(lemmatize)

In [32]:
df.head()

,review_body,good_product,text
0,Jamás me llegó y el vendedor nunca contacto co...,0,jamás vendedor contacto intentar él 2
1,Pone que son 4 piezas y la realidad es que es ...,0,poner 4 pieza realidad 3 pieza
2,Saltan los plomos al tercer día de uso. A devo...,0,saltar plomos tercer . devolver
3,No me ha gustado de hecho la devolví . Súper p...,0,gustado devolví . súper pesado manejar
4,"Por más que busque, no le encuentro el agujero...",0,"buscar , encuentro agujero meter chuch . instr..."


## Modelo

In [33]:
sentences_bad_product = df[df.good_product==0]['text'].to_list()

tam_vocab = 30

tokenizer_bad = Tokenizer(num_words=tam_vocab)
tokenizer_bad.fit_on_texts(sentences_bad_product)

In [34]:
# tokenizer_bad.word_index

In [35]:
all_texts = df['text'].to_list()

tokenized_bad = tokenizer_bad.texts_to_sequences(all_texts)

In [36]:
print(len(all_texts))
print(len(tokenized_bad))

10634
10634


In [37]:
# tokekinzed_bad

In [38]:
data = []
for tokens in tokenized_bad:
    row = [1 if i in tokens else 0 for i in range(1, tam_vocab+1)]
    data.append(row)
column_names = [f"b{i}" for i in range(1, tam_vocab+1)]

df_bad = pd.DataFrame(data, columns=column_names)
df_bad.head()

,b1,b2,b3,b4,b5,b6,b7,b8,b9,b10,...,b21,b22,b23,b24,b25,b26,b27,b28,b29,b30
0,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


## Good products

In [39]:
sentences_good_product = df[df.good_product==1]['text'].to_list()

tam_vocab = 30

tokenizer_good = Tokenizer(num_words=tam_vocab)
tokenizer_good.fit_on_texts(sentences_good_product)

In [40]:
# tokenizer_good.word_index

In [41]:
all_texts = df['text'].to_list()

tokenized_good = tokenizer_good.texts_to_sequences(all_texts)

In [42]:
# tokenized_good

In [43]:
data = []
for tokens in tokenized_good:
    row = [1 if i in tokens else 0 for i in range(1, tam_vocab+1)]
    data.append(row)
column_names = [f"g{i}" for i in range(1, tam_vocab+1)]

df_good = pd.DataFrame(data, columns=column_names)
df_good.head()

,g1,g2,g3,g4,g5,g6,g7,g8,g9,g10,...,g21,g22,g23,g24,g25,g26,g27,g28,g29,g30
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
print(len(df_good))
print(len(df_bad))
df_final = pd.merge(df_bad, df_good, left_index=True, right_index=True, how='inner')
print(len(df_final))

10634
10634
10634


In [45]:
df['good_product']

0        0
1        0
2        0
3        0
4        0
        ..
26957    1
26958    1
26959    1
26960    1
26961    1
Name: good_product, Length: 10634, dtype: int64

In [46]:
df_final['y'] = df['good_product'].values
df_final.tail()

,b1,b2,b3,b4,b5,b6,b7,b8,b9,b10,...,g22,g23,g24,g25,g26,g27,g28,g29,g30,y
10629,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
10630,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
10631,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
10632,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
10633,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [47]:
from sklearn.tree import DecisionTreeClassifier

X = df_final.drop(columns='y')
y = df_final.y

model = DecisionTreeClassifier(random_state=99)
model.fit(X, y)
model.score(X, y)

0.8317660334775249

# N-Grams

Son combinaciones contiguas de n palabras de una secuencia de text (cafe caliente)

In [48]:
from nltk.util import ngrams

def generate_ngrams(text, n):
    tokens = text.split()
    n_grams = ngrams(tokens, n)
    return list(n_grams)

txt = "esto es un texto de ejemplo en natural language processing"
generate_ngrams(txt, n=3)

[('esto', 'es', 'un'),
 ('es', 'un', 'texto'),
 ('un', 'texto', 'de'),
 ('texto', 'de', 'ejemplo'),
 ('de', 'ejemplo', 'en'),
 ('ejemplo', 'en', 'natural'),
 ('en', 'natural', 'language'),
 ('natural', 'language', 'processing')]

# TF-IDF Algorithm

In [49]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/eduardofc/data/main/amazon_home.csv")
df['review_body'] = df['review_body'].str.replace('[^a-zA-ZñÑáéíóúü .,]', '', regex=True)
df.head()

,stars,review_body,review_title,product_category
0,1,Jamás me llegó y el vendedor nunca contacto co...,Jamás me llegó,home
1,1,Pone que son piezas y la realidad es que es d...,Mala descripción,home
2,1,Saltan los plomos al tercer día de uso. A devo...,Saltan los plomos al utilizarla,home
3,1,No me ha gustado de hecho la devolví . Súper p...,No merece la pena,home
4,1,"Por más que busque, no le encuentro el agujero...",No tiene agujero para rellenar la piñata,home


Scikit learn tiene el TFIDF

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words=list(stopwords_list))

reviews = df.review_body.values
tfidf_matrix = vectorizer.fit_transform(reviews).toarray()
words = vectorizer.get_feature_names_out()

In [51]:
print(words)
print(len(words))
print(tfidf_matrix.shape)
tfidf_matrix

['aa' 'aaa' 'abajo' ... 'útil' 'útiles' 'útlimo']
21344
(26962, 21344)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [52]:
pd.set_option('display.max_colwidth', 100)

df_tfidf = pd.DataFrame(tfidf_matrix, columns=words)
df_tfidf.head()

,aa,aaa,abajo,abalorios,abandonan,abandonando,abanico,abanicos,abarata,abaratar,...,óxido,últimamente,única,únicamente,únicas,único,únicos,útil,útiles,útlimo
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
df_tfidf.sum()

aa            3.929498
aaa           2.329155
abajo        37.411509
abalorios     1.333735
abandonan     0.455185
               ...    
único        93.971243
únicos        1.085936
útil         87.823833
útiles       22.713322
útlimo        0.164957
Length: 21344, dtype: float64

In [54]:
df.head()

,stars,review_body,review_title,product_category
0,1,Jamás me llegó y el vendedor nunca contacto conmigo a pesar de intentarlo veces,Jamás me llegó,home
1,1,Pone que son piezas y la realidad es que es de piezas,Mala descripción,home
2,1,Saltan los plomos al tercer día de uso. A devolver,Saltan los plomos al utilizarla,home
3,1,No me ha gustado de hecho la devolví . Súper pesada para manejar,No merece la pena,home
4,1,"Por más que busque, no le encuentro el agujero para meter las chuches. En las instrucciones dice...",No tiene agujero para rellenar la piñata,home


In [55]:
df_tfidf2 = pd.DataFrame(df_tfidf.sum().reset_index())
df_tfidf2.columns = ['word', 'sum_tfidf']
df_tfidf2.sort_values('sum_tfidf', ascending=False).head(20)

,word,sum_tfidf
3001,calidad,727.585123
15819,precio,561.840973
16117,producto,522.990360
8392,esperaba,326.269475
12315,luz,310.550959
15000,perfecto,272.686278
9458,foto,270.057330
2575,bonito,269.649991
12089,llegado,264.419491
5347,cumple,253.645066


# Ejercicio (no-supervisado)

In [56]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, random_state=99)

In [58]:
vectorizer = TfidfVectorizer(stop_words=list(stopwords_list))

X = vectorizer.fit_transform(reviews)

In [61]:
kmeans.fit(X)

KMeans(n_clusters=5, random_state=99)

In [66]:
pd.set_option('display.max_colwidth', 300)

clusters = kmeans.labels_

df_kmeans = pd.DataFrame(zip(reviews, clusters), columns=['review', 'cluster'])
df_kmeans.head()

,review,cluster
0,Jamás me llegó y el vendedor nunca contacto conmigo a pesar de intentarlo veces,2
1,Pone que son piezas y la realidad es que es de piezas,2
2,Saltan los plomos al tercer día de uso. A devolver,2
3,No me ha gustado de hecho la devolví . Súper pesada para manejar,2
4,"Por más que busque, no le encuentro el agujero para meter las chuches. En las instrucciones dice que viene tapada con una pegatina transparente, pero la mia no la tiene.",2


In [67]:
df_kmeans.groupby('cluster').size()

cluster
0     2114
1      868
2    21064
3     1251
4     1665
dtype: int64

In [70]:
df_kmeans[df_kmeans.cluster==4]

,review,cluster
16,"Muy mala calidad, se despega enseguida",4
27,La caja es demasiado pequeña para el precio que tiene,4
48,las cajas vienes muy deformadas mala calidad precio me esperaba otra cosa no son igual que en la foto gracias,4
59,Precio elevado. He visto el producto en otros lados mucho más barato. La calidad es regular. No recomiendo su compra,4
76,"No me gusto nada la calidad del producto, mis expectativas eran otras acerca de la calidad,pensé que era más suave",4
...,...,...
26875,Me ha encantado el diseño que tiene con el círculo central más grande que el resto. Material de calidad por un precio económico,4
26909,"Es preciosa, muy alegre y de buena calidad",4
26933,"Perfecta, buena calidad.",4
26946,"Vienen luces solares, cargan bastante rápido y por la noche su iluminación es ambiente y buena en forma de estrella, calidad precio está bien",4
